## Import Modul pendukung

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import xgboost as xgb
import os

## Import Dataset untuk Sample Machine Learningnya

In [5]:
df = pd.read_csv("C:\\Users\\LENOVO\\Documents\\Machine Learning\\Weather Type Classification\\weather_classification_data.csv")
df

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,overcast,1003.15,1,Summer,1.0,mountain,Rainy
13196,-1.0,76,3.5,23.0,cloudy,1067.23,1,Winter,6.0,coastal,Snowy
13197,30.0,77,5.5,28.0,overcast,1012.69,3,Autumn,9.0,coastal,Cloudy
13198,3.0,76,10.0,94.0,overcast,984.27,0,Winter,2.0,inland,Snowy


## Inisialisasi untuk data yang berbentuk text / kata
Mengubah Table yang berisi text / kata agar nantinya dapat di baca oleh sistem. mengubah pada semua table yang ada text / kata nya, ke bentuk numerik (0,1,2-seterusnya)

In [4]:
encoder = LabelEncoder()

### Melakukan Transformasi untuk bagian Feature Data 

In [6]:
df['Cloud Cover']   = encoder.fit_transform(df['Cloud Cover'])
df['Location']      = encoder.fit_transform(df['Location'])
df['Season']        = encoder.fit_transform(df['Season'])

### Melakukan Transformasi untuk bagian Target Data

In [7]:
y_target = encoder.fit_transform(df["Weather Type"])

### Melakukan pemisahan dataset dari Target Datanya

In [12]:
dataset = df[[i for i in df.columns[:-1]]]
dataset

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location
0,14.0,73,9.5,82.0,3,1010.82,2,3,3.5,1
1,39.0,96,8.5,71.0,3,1011.43,7,1,10.0,1
2,30.0,64,7.0,16.0,0,1018.72,5,1,5.5,2
3,38.0,83,1.5,82.0,0,1026.25,7,1,1.0,0
4,27.0,74,17.0,66.0,2,990.67,1,3,2.5,2
...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,2,1003.15,1,2,1.0,2
13196,-1.0,76,3.5,23.0,1,1067.23,1,3,6.0,0
13197,30.0,77,5.5,28.0,2,1012.69,3,0,9.0,0
13198,3.0,76,10.0,94.0,2,984.27,0,3,2.0,1


### Melakukan pembagian dataset
Pembagian untuk **Training Data** dan **Test Data**

In [13]:
x_train, x_test, y_train, y_test = train_test_split(dataset, 
                                                    y_target,
                                                    test_size=0.2,
                                                    random_state=50)

## Menentukan Parameter untuk membuat Model ML

- **objective: 'multi:softmax'**
  Untuk menentukan jenis klasifikasi. **'multi:softmax'** berarti model menggunakan softmax untuk klasifikasi multikelas.
- **num_class: len(df.columns[:-1])**
  Untuk menentukan jumlah kelas dalam masalah klasifikasi. **df.columns[:-1]** mengambil semua kolom kecuali kolom terakhir (biasanya label/target).
- **eta: 0.01**
  Untuk Learning rate, juga dikenal sebagai step size shrinkage. Nilai kecil (misalnya 0.01) membantu model belajar lebih lambat tetapi lebih stabil.
- **max_depth: 6**
  Untuk menentukan kedalaman maksimum pohon keputusan dalam model. Kedalaman yang lebih besar dapat meningkatkan kapasitas model tetapi juga meningkatkan risiko overfitting.

In [14]:
parameters = {
    'objective':'multi:softmax',
    'num_class': len(df.columns[:-1]),
    'eta': 0.01,
    'max_depth': 6,
}

C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:723: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


## Melakukan pembuatan Model Machine Learning dengan XGBClassifier

model = xgb.XGBClassifier(
    parameters
)

In [15]:
model.fit(x_train, y_train)
model.save_model("XGBClassifier.json")

In [16]:
pred = model.predict(x_test)

# Hitung akurasi prediksi
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

pred = encoder.inverse_transform(pred)
y_test = encoder.inverse_transform(y_test)

Accuracy: 0.9159090909090909


In [19]:
database_salah = []
database_benar = []
db = []

for prediction, target in zip(pred, y_test):
    data = f'Prediction: {prediction}\tTarget: {target}'
    if prediction != target:
        db.append('Salah')
        database_salah.append(data)
    else:
        db.append('Benar')
        database_benar.append(data)

data = zip(
    np.array(pred),
    np.array(y_test),
    np.array(db)
)

dfPrediction = pd.DataFrame(data, columns=['Prediction', 'Target','Salah/Benar'])
print(f'Jumalah data Precision salah: {len(database_salah)}')
print(f'Jumalah data Precision benar: {len(database_benar)}')

Jumalah data Precision salah: 222
Jumalah data Precision benar: 2418
